# ACS Backup
This script downloads and saves CT (state, county, planning region, town) ACS data for user-specified years and table IDs. A full list of table IDs can be downloaded on the [Census Bureau's website](https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.html).

In [ ]:
import censusdis.data as ced
from censusdis.impl.fetch import CensusApiException
import pandas as pd
import time
import os

## Specify Inputs

In [ ]:
API_KEY = ''

In [ ]:
years = years = list(range(2010, 2024))

In [ ]:
# Table IDs of interest
tables = []

## Functions to Download Data

Map the table type (profile, supplemental estimate, subject, other) to the correct URL.

In [ ]:
def get_dataset(table):
    if table.startswith('D'):
        return 'acs/acs5/profile'
    elif table.startswith('K'):
        return 'acs/acs5/acsse'
    elif table.startswith('S'):
        return 'acs/acs5/subject'
    else:
        return 'acs/acs5'

Get estimates and margins of errors for specified years and tables.

In [ ]:
def get_data(years, tables):
    
    for table in tables:
        dataset = get_dataset(table)
        start_time = time.time() # Start timer
        all_years_combined = []

        for year in years:
            try:
                # Get variable codes
                df_variables = ced.variables.all_variables(dataset, year, table)
                variables = [variable for variable in df_variables['VARIABLE']]
                
                # Add MOE variables
                all_variables = []
                for var in variables:
                    all_variables.append(var)
                    if var.endswith('E') and var != 'NAME': # Check if it's an estimate variable
                        moe_var = var[:-1] + 'M' # Replace 'E' with 'M'
                        all_variables.append(moe_var)
        
                # Download data for state, county/cog, and town
                state_df = ced.download(dataset, year, all_variables, state="09", api_key=API_KEY)
                county_df = ced.download(dataset, year, all_variables, state="09", county="*", api_key=API_KEY)
                town_df = ced.download(dataset, year, all_variables, state="09", county_subdivision="*", api_key=API_KEY)
                
                # Add current year before concatenating (fragmentation warning adding the col after concatenating)
                for df in [state_df, county_df, town_df]:
                    df['year'] = year

                # Combine data for the current year and add year col
                combined_df = pd.concat([state_df, county_df, town_df], ignore_index=True)
                
                # Append to the list for all years
                all_years_combined.append(combined_df)

            except CensusApiException as e:
                print(f"{table}: No data for {year}")

        # Final combined df for the current table
        if all_years_combined: # Only concatenate if there's data
            final_combined_df = pd.concat(all_years_combined, ignore_index=True)
            
            # Reorder columns
            priority_columns = ['year', 'NAME', 'GEO_ID', 'STATE', 'COUNTY', 'COUNTY_SUBDIVISION']
            existing_priority_cols = [col for col in priority_columns if col in final_combined_df.columns]
            other_columns = [col for col in final_combined_df.columns if col not in existing_priority_cols]
            
            # Apply new column order
            final_combined_df = final_combined_df[existing_priority_cols + other_columns]
            
            # Output to CSV
            output_filename = f"{table}.csv"
            final_combined_df.to_csv(f"2010 to 2023 5-Year ACS Data/{output_filename}", index=False)
        
        # Calculate the elapsed time
        elapsed_time = (time.time() - start_time) / 60  # Convert to minutes
        print(f"{table} complete ({elapsed_time:.1f} minutes)")


In [ ]:
get_data(years, tables)